In [2]:
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

# Download NLTK resources
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# Read the dataset
df = pd.read_csv("clean_data.csv")

In [4]:
df.head()

,ID,Dates Review,Day Review,Month Review,Month Review Number,Year Review,Verified,Customer Name,Month Flown,Month Flown Number,...,Seat Type,Route,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Value For Money,Wifi & Connectivity,Recommended,Review
0,0,2024-02-23,23,February,2,2024,True,Peter Hansell,September,9.0,...,Economy Class,Pisa to London Heathrow,NaN,NaN,NaN,NaN,1,NaN,no,That was supposed to be my flight but it was c...
1,1,2024-02-21,21,February,2,2024,True,Caleb Lowe,February,2.0,...,Business Class,Malaga to London Heathrow,5.0,5.0,5.0,4.0,4,NaN,yes,Have no fear when your BA flight is operated b...
2,2,2024-02-17,17,February,2,2024,False,Kevin Statham,February,2.0,...,Economy Class,Phoenix to Lisbon via London,1.0,5.0,3.0,1.0,1,1.0,no,We have flown BA five times from western USA t...
3,3,2024-02-17,17,February,2,2024,True,C Horton,February,2.0,...,Economy Class,London to Istanbul,3.0,1.0,1.0,2.0,2,NaN,no,London Heathrow to Istanbul at the start of th...
4,4,2024-02-16,16,February,2,2024,False,Anthony C Gallagher,January,1.0,...,Premium Economy,"Austin, TX to Shannon, Ireland via Austin",3.0,5.0,3.0,4.0,2,NaN,no,"Jan 30th, I booked a last-minute, transatlanti..."


In [5]:
df = df[['Review','Value For Money', 'Recommended']]

In [6]:
df.head()

,Review,Value For Money,Recommended
0,That was supposed to be my flight but it was c...,1,no
1,Have no fear when your BA flight is operated b...,4,yes
2,We have flown BA five times from western USA t...,1,no
3,London Heathrow to Istanbul at the start of th...,2,no
4,"Jan 30th, I booked a last-minute, transatlanti...",2,no
